## 1. Dataset preparation

In [1]:
import json
from dataset_dic import formatted_data
ds_formatted = formatted_data()

In [2]:
ds_formatted[0]

{'messages': [{'role': 'system',
   'content': "You are a robot assistant that can extract keywords from user's demands and help us save the keywords and corresponding parameters into a dictionary format."},
  {'role': 'user',
   'content': 'Move the cup forward 10 cm relative to the box'},
  {'role': 'assistant',
   'content': {'what': 'cup',
    'how': 'move',
    'location': {'precise': 'False',
     'relation': 'box',
     'direction': 'forward',
     'distance': '10 cm'}}}]}

In [3]:
len(ds_formatted)

14

In [8]:
import random
random.shuffle(ds_formatted)

In [9]:
ds_train = ds_formatted[:10]
ds_val = ds_formatted[10:]

In [17]:
# preprocess content: dict to str
for i in range(len(ds_formatted)):
    init_dic = ds_formatted[i]['messages'][2]['content']
    dic2json = json.dumps(init_dic, indent=4)
    ds_formatted[i]['messages'][2]['content'] = dic2json

In [18]:
import json

with open('train.jsonl', 'w') as f:
    for line in ds_train:
        json.dump(line, f)
        f.write('\n')

with open('val.jsonl', 'w') as f:
    for line in ds_val:
        json.dump(line, f)
        f.write('\n')

In [18]:
ls

dataset_dic.py                 our_fine_tune_model.ipynb  train.jsonl
fine_tune_gpt_3_5_turbo.ipynb  __pycache__/               val.jsonl


In [19]:
!head -n 2 train.jsonl

{"messages": [{"role": "system", "content": "You are a robot assistant that can extract keywords from user's demands and help us save the keywords and corresponding parameters into a dictionary format."}, {"role": "user", "content": "Slide the mouse pad closer to the keyboard"}, {"role": "assistant", "content": {"what": "mouse pad", "how": "move", "location": {"precise": "False", "relation": "keyboard", "direction": "closer", "distance": "None"}}}]}
{"messages": [{"role": "system", "content": "You are a robot assistant that can extract keywords from user's demands and help us save the keywords and corresponding parameters into a dictionary format."}, {"role": "user", "content": "Slide the vase towards the center of the table"}, {"role": "assistant", "content": {"what": "vase", "how": "move", "location": {"precise": "False", "relation": "table center", "direction": "toward", "distance": "None"}}}]}


## 2. Upload data

In [19]:
import openai

openai.api_key = 'openai-api'

# pip install openai==0.28 # download the old version and then restart kernel

In [20]:
# Upload training data
train = openai.File.create(
  file=open("train.jsonl", "rb"),
  purpose='fine-tune'
)
train

<File file id=file-1dFLfKGOFZO4mFcvX95dRzno at 0x7fcf03bfd530> JSON: {
  "object": "file",
  "id": "file-1dFLfKGOFZO4mFcvX95dRzno",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 6108,
  "created_at": 1715079745,
  "status": "processed",
  "status_details": null
}

In [21]:
train_id = train['id']

In [22]:
# Upload validation data
val = openai.File.create(
  file=open("val.jsonl", "rb"),
  purpose='fine-tune'
)
val

<File file id=file-vrtMeOxKgQWu8JKKIbdvYUwZ at 0x7fcee9e27560> JSON: {
  "object": "file",
  "id": "file-vrtMeOxKgQWu8JKKIbdvYUwZ",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 2479,
  "created_at": 1715079749,
  "status": "processed",
  "status_details": null
}

In [23]:
val_id = val['id']

## 3. Fine tune

In [24]:
# Create a fine-tuned model
response = openai.FineTuningJob.create(
    training_file=train_id,
    validation_file=val_id,
    model="gpt-3.5-turbo"
    )
response

<FineTuningJob fine_tuning.job id=ftjob-WDk5nbXleKqrubR1kn4FLiks at 0x7fcee82ae610> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-WDk5nbXleKqrubR1kn4FLiks",
  "model": "gpt-3.5-turbo-0125",
  "created_at": 1715079757,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-zTbtP7D5muX7kI6oqtsYYww1",
  "result_files": [],
  "status": "validating_files",
  "validation_file": "file-vrtMeOxKgQWu8JKKIbdvYUwZ",
  "training_file": "file-1dFLfKGOFZO4mFcvX95dRzno",
  "hyperparameters": {
    "n_epochs": "auto",
    "batch_size": "auto",
    "learning_rate_multiplier": "auto"
  },
  "trained_tokens": null,
  "error": {},
  "user_provided_suffix": null,
  "seed": 785827168,
  "estimated_finish": null,
  "integrations": []
}

In [25]:
job_id = response['id']

In [26]:
# Retrieve the state of a fine-tune
response = openai.FineTuningJob.retrieve(job_id)
response

<FineTuningJob fine_tuning.job id=ftjob-WDk5nbXleKqrubR1kn4FLiks at 0x7fcee82d6d40> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-WDk5nbXleKqrubR1kn4FLiks",
  "model": "gpt-3.5-turbo-0125",
  "created_at": 1715079757,
  "finished_at": 1715080077,
  "fine_tuned_model": "ft:gpt-3.5-turbo-0125:personal::9MCxqkKh",
  "organization_id": "org-zTbtP7D5muX7kI6oqtsYYww1",
  "result_files": [
    "file-bWlsMpamZOKUzxpDAK13d95z"
  ],
  "status": "succeeded",
  "validation_file": "file-vrtMeOxKgQWu8JKKIbdvYUwZ",
  "training_file": "file-1dFLfKGOFZO4mFcvX95dRzno",
  "hyperparameters": {
    "n_epochs": 10,
    "batch_size": 1,
    "learning_rate_multiplier": 2
  },
  "trained_tokens": 11480,
  "error": {},
  "user_provided_suffix": null,
  "seed": 785827168,
  "estimated_finish": null,
  "integrations": []
}

In [27]:
# List up to 10 events from a fine-tuning job
response = openai.FineTuningJob.list_events(id=job_id, limit=10)
response

<OpenAIObject list at 0x7fcee82d75b0> JSON: {
  "object": "list",
  "data": [
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-SWM7PjIBtz4nPl40LfBxf4qk",
      "created_at": 1715080082,
      "level": "info",
      "message": "The job has successfully completed",
      "data": {},
      "type": "message"
    },
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-asfH2Alk7XrYgM06Fj3WVT04",
      "created_at": 1715080079,
      "level": "info",
      "message": "New fine-tuned model created: ft:gpt-3.5-turbo-0125:personal::9MCxqkKh",
      "data": {},
      "type": "message"
    },
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-oPnVb0lOABVo7IjEbZLR1dMh",
      "created_at": 1715080078,
      "level": "info",
      "message": "Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:personal::9MCxq16B:ckpt-step-90",
      "data": {},
      "type": "message"
    },
    {
      "object": "fine_tuning.job.event",
      "id"

In [28]:
events = response["data"]
events.reverse()

for event in events:
    print(event["message"])

Step 95/100: training loss=0.00, validation loss=0.42
Step 96/100: training loss=0.00, validation loss=0.02
Step 97/100: training loss=0.00, validation loss=0.00
Step 98/100: training loss=0.00, validation loss=0.00
Step 99/100: training loss=0.00, validation loss=0.41
Step 100/100: training loss=0.00, validation loss=0.02, full validation loss=0.02
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:personal::9MCxpdtR:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:personal::9MCxq16B:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:personal::9MCxqkKh
The job has successfully completed


In [29]:
response = openai.FineTuningJob.retrieve(job_id)
response

<FineTuningJob fine_tuning.job id=ftjob-WDk5nbXleKqrubR1kn4FLiks at 0x7fcee82d7100> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-WDk5nbXleKqrubR1kn4FLiks",
  "model": "gpt-3.5-turbo-0125",
  "created_at": 1715079757,
  "finished_at": 1715080077,
  "fine_tuned_model": "ft:gpt-3.5-turbo-0125:personal::9MCxqkKh",
  "organization_id": "org-zTbtP7D5muX7kI6oqtsYYww1",
  "result_files": [
    "file-bWlsMpamZOKUzxpDAK13d95z"
  ],
  "status": "succeeded",
  "validation_file": "file-vrtMeOxKgQWu8JKKIbdvYUwZ",
  "training_file": "file-1dFLfKGOFZO4mFcvX95dRzno",
  "hyperparameters": {
    "n_epochs": 10,
    "batch_size": 1,
    "learning_rate_multiplier": 2
  },
  "trained_tokens": 11480,
  "error": {},
  "user_provided_suffix": null,
  "seed": 785827168,
  "estimated_finish": null,
  "integrations": []
}

In [30]:
model_id = response["fine_tuned_model"]

## 4. Inference

In [31]:
test_messages = [
    {"role": "system", "content": "You are a robot assistant that can extract keywords from user's demands and help us save the keywords and corresponding parameters into a dictionary format."},
    {"role": "user", "content": "move the box to (10,10) point"}
  ]

In [32]:
completion = openai.ChatCompletion.create(
  model=model_id,
  messages=test_messages
)

In [33]:
print(completion.choices[0].message)

{
  "role": "assistant",
  "content": "\"{\\n    \\\"what\\\": \\\"box\\\",\\n    \\\"how\\\": \\\"move\\\",\\n    \\\"location\\\": {\\n        \\\"precise\\\": \\\"False\\\",\\n        \\\"relation\\\": \\\"None\\\",\\n        \\\"old\\\": \\\"None\\\",\\n        \\\"new\\\": \\\"(10,10)\\\"\\n    }\\n}\""
}


In [42]:
gpt_reply = completion.choices[0].message
reply_dict = json.loads(json.loads(gpt_reply['content']))
reply_dict
# type(gpt_reply['content'])

{'what': 'box',
 'how': 'move',
 'location': {'precise': 'False',
  'relation': 'None',
  'old': 'None',
  'new': '(10,10)'}}

In [43]:
reply_dict['what']

'box'

In [44]:
completion1 = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=test_messages
)

In [45]:
print(completion1.choices[0].message)

{
  "role": "assistant",
  "content": "Keywords: move, box, coordinates\nParameters: \n- Object: box\n- Destination coordinates: (10, 10)"
}
